In [58]:
import pandas as pd
import re
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

In [59]:
# Tool to delete unicode codes
def unicodetoascii(text):
    TEXT = (text.
    		replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').#
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")")
                 )
    return TEXT

#Reading CSV file
dataset=pd.read_csv('tweets3.csv', sep=',',quotechar='"',header=None,)
#Replacing Unicode characters
dataset[2]=dataset[2].apply(unicodetoascii)
#Deleting Urls
delete_urls = lambda x:re.sub(r'http\S+', '', x)
dataset[2] = dataset[2].apply(delete_urls)
#Deleting "
delete_quotes = lambda x:re.sub(r'"', '', x)
dataset[2] = dataset[2].apply(delete_quotes)
#Deleting b
delete_b = lambda x:re.sub(r'b\'', '', x)
dataset[2] = dataset[2].apply(delete_b)

#Datasets
X=dataset[2].values
Y=dataset[3].values


In [60]:
#Data split
x_train, x_test ,y_train,y_test = train_test_split(X,Y, test_size=0.3)

In [54]:
#Machine Learning NB
text_clf_NB = Pipeline([('vect', CountVectorizer(stop_words='english')),('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])
text_clf_NB = text_clf.fit(x_train, y_train)

# Performance
y_predicted = text_clf_NB.predict(x_test)
np.mean(y_predicted == y_test)
# 78% accuracy

0.78502018238899685

In [55]:
#Machine Learning SVM
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english')),
                   ('tfidf', TfidfTransformer()),
                   ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42)), ])

_ = text_clf_svm.fit(x_train,y_train)
predicted_svm = text_clf_svm.predict(x_test)
np.mean(predicted_svm == y_test)
# 71% accuracy

/home/loko/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:73: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.69561967409179248

In [56]:
#Grid search NB
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False),'clf__alpha': (1e-2, 1e-3)}
gs_clf = GridSearchCV(text_clf_NB, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(x_train,y_train)
print(gs_clf.best_score_)
gs_clf.best_params_
#78% accuracy

0.785132306772


{'clf__alpha': 0.01, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}

In [57]:
#Grid search SVM
from sklearn.model_selection import GridSearchCV
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False),'clf-svm__alpha': (1e-2, 1e-3)}
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(x_train,y_train)
print(gs_clf_svm.best_score_)
gs_clf_svm.best_params_
#71% accuracy

0.709261474062


{'clf-svm__alpha': 0.001, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}